In [1]:

import pandas as pd

import typing as tt
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import product
from tqdm import tqdm

In [2]:
df=pd.read_csv('../EUR_USD_2020_M5.csv')

In [3]:
df=df.set_index('time')

In [4]:
class EMD():

    def __init__(self,data):

        self.data=data
        self.possible_strats={'strategy_desc':'Strategy EMD (EmpiricalModeDecomposition by J.Ehlers) change position above below given levels',
                            'm_a_ap_b_av':{'name':'mean_above_avg_peak_below_avg_valley',
                            'positions':{'buy':'mean_above_avg_peak','sell':'mean_below_avg_valley'}},
                            'm_a_av_b_ap':{'name':'mean_above_avg_valley_below_avg_peak',
                                         'positions':{'buy':'mean_above_avg_vallet','sell':'mean_below_avg_peak'}}}
                            

        
                                           
        self.params_range={'period':[25,40,55],
                            'delta':[0.4,0.5,0.6],
                            'fraction':[0.15,0.25,0.35],
                             'pv_window':[30,40,50]}

    def create_params_combs(self):

        return list(product(*self.params_range.values()))

    def cosine(self,x:float)->float:

        return np.cos(np.radians(x))

    def average(self,arr, window):

        if len(arr)==0:
            return 0
        else:
            calc_arr=arr[-window:]
            cumsum=np.cumsum(calc_arr)
            return cumsum[-1]/window

    def calc_indicator(self, period, delta,fraction,pv_window):
        self.data['Price']=(self.data['h']+self.data['l'])/2
        
        beta=self.cosine(360/period)
        gamma=1/self.cosine(720*delta/period)
        alpha=gamma-np.sqrt(gamma*gamma-1)
        bp_arr=[]
        peak_arr=[]
        valley_arr=[]
        mean_arr=[]
        avgpeak_arr=[]
        avgvalley_arr=[]

        avgpeak_colname=f'AvgPeak_{period}_{delta}_{fraction}_{pv_window}'
        avgvalley_colname=f'AvgValley_{period}_{delta}_{fraction}_{pv_window}'
        mean_colname=f'Mean_{period}_{delta}_{fraction}_{pv_window}'

        for i in range(len(self.data)):

            Price=self.data['Price'].iloc[i]
            Price_2=self.data['Price'].iloc[i-2] if i-2>=0 else self.data['Price'].iloc[0]
            BP_1=bp_arr[i-1] if i-1 >=0 else 0
            BP_2=bp_arr[i-2] if i-2>=0 else 0
            Peak_1=peak_arr[i-1] if i-1>=0 else 0
            Valley_1=valley_arr[i-1] if i-1>=0 else 0

            BP=0.5*(1-alpha)*(Price-Price_2)+beta*(1+alpha)*BP_1-alpha*BP_2

            Mean=self.average(bp_arr,2*period)
            Peak=Peak_1
            Valley=Valley_1

            AvgPeak=self.average(peak_arr,pv_window)
            AvgValley=self.average(valley_arr,pv_window)

            bp_arr.append(BP)
            peak_arr.append(Peak)
            valley_arr.append(Valley)
            mean_arr.append(Mean)
            avgpeak_arr.append(AvgPeak)
            avgvalley_arr.append(AvgValley)

        self.data[avgpeak_colname]=avgpeak_arr
        self.data[avgvalley_colname]=avgvalley_arr
        self.data[mean_colname]=mean_arr

        


        
        
                            

In [5]:
emd=EMD(df)

In [6]:
len(emd.create_params_combs())

81

In [7]:
combs=emd.create_params_combs()

In [8]:
combs

[(25, 0.4, 0.15, 30),
 (25, 0.4, 0.15, 40),
 (25, 0.4, 0.15, 50),
 (25, 0.4, 0.25, 30),
 (25, 0.4, 0.25, 40),
 (25, 0.4, 0.25, 50),
 (25, 0.4, 0.35, 30),
 (25, 0.4, 0.35, 40),
 (25, 0.4, 0.35, 50),
 (25, 0.5, 0.15, 30),
 (25, 0.5, 0.15, 40),
 (25, 0.5, 0.15, 50),
 (25, 0.5, 0.25, 30),
 (25, 0.5, 0.25, 40),
 (25, 0.5, 0.25, 50),
 (25, 0.5, 0.35, 30),
 (25, 0.5, 0.35, 40),
 (25, 0.5, 0.35, 50),
 (25, 0.6, 0.15, 30),
 (25, 0.6, 0.15, 40),
 (25, 0.6, 0.15, 50),
 (25, 0.6, 0.25, 30),
 (25, 0.6, 0.25, 40),
 (25, 0.6, 0.25, 50),
 (25, 0.6, 0.35, 30),
 (25, 0.6, 0.35, 40),
 (25, 0.6, 0.35, 50),
 (40, 0.4, 0.15, 30),
 (40, 0.4, 0.15, 40),
 (40, 0.4, 0.15, 50),
 (40, 0.4, 0.25, 30),
 (40, 0.4, 0.25, 40),
 (40, 0.4, 0.25, 50),
 (40, 0.4, 0.35, 30),
 (40, 0.4, 0.35, 40),
 (40, 0.4, 0.35, 50),
 (40, 0.5, 0.15, 30),
 (40, 0.5, 0.15, 40),
 (40, 0.5, 0.15, 50),
 (40, 0.5, 0.25, 30),
 (40, 0.5, 0.25, 40),
 (40, 0.5, 0.25, 50),
 (40, 0.5, 0.35, 30),
 (40, 0.5, 0.35, 40),
 (40, 0.5, 0.35, 50),
 (40, 0.6,

In [11]:
for c in tqdm(combs[:5]):

    emd.calc_indicator(*c)

100%|██████████| 5/5 [00:15<00:00,  3.00s/it]


In [10]:
emd.data

,o,h,l,c,volume,complete,Price,AvgPeak_25_0.4_0.15_30,AvgValley_25_0.4_0.15_30,Mean_25_0.4_0.15_30,...,Mean_25_0.4_0.15_40,AvgPeak_25_0.4_0.15_50,AvgValley_25_0.4_0.15_50,Mean_25_0.4_0.15_50,AvgPeak_25_0.4_0.25_30,AvgValley_25_0.4_0.25_30,Mean_25_0.4_0.25_30,AvgPeak_25_0.4_0.25_40,AvgValley_25_0.4_0.25_40,Mean_25_0.4_0.25_40
time,,,,,,,,,,,,,,,,,,,,,
2020-01-01 22:00:00+00:00,1.12124,1.12146,1.12124,1.12132,9,True,1.121350,0.0,0.0,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
2020-01-01 22:05:00+00:00,1.12137,1.12137,1.12125,1.12125,5,True,1.121310,0.0,0.0,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
2020-01-01 22:10:00+00:00,1.12129,1.12129,1.12125,1.12125,2,True,1.121270,0.0,0.0,-7.330269e-08,...,-7.330269e-08,0.0,0.0,-7.330269e-08,0.0,0.0,-7.330269e-08,0.0,0.0,-7.330269e-08
2020-01-01 22:15:00+00:00,1.12135,1.12155,1.12134,1.12146,12,True,1.121445,0.0,0.0,-3.488964e-07,...,-3.488964e-07,0.0,0.0,-3.488964e-07,0.0,0.0,-3.488964e-07,0.0,0.0,-3.488964e-07
2020-01-01 22:20:00+00:00,1.12143,1.12154,1.12143,1.12154,4,True,1.121485,0.0,0.0,-5.265835e-07,...,-5.265835e-07,0.0,0.0,-5.265835e-07,0.0,0.0,-5.265835e-07,0.0,0.0,-5.265835e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 21:35:00+00:00,1.22201,1.22207,1.22196,1.22206,35,True,1.222015,0.0,0.0,-4.285440e-05,...,-4.285440e-05,0.0,0.0,-4.285440e-05,0.0,0.0,-4.285440e-05,0.0,0.0,-4.285440e-05
2020-12-31 21:40:00+00:00,1.22207,1.22207,1.22188,1.22190,69,True,1.221975,0.0,0.0,-4.251157e-05,...,-4.251157e-05,0.0,0.0,-4.251157e-05,0.0,0.0,-4.251157e-05,0.0,0.0,-4.251157e-05
2020-12-31 21:45:00+00:00,1.22192,1.22196,1.22174,1.22174,72,True,1.221850,0.0,0.0,-4.314044e-05,...,-4.314044e-05,0.0,0.0,-4.314044e-05,0.0,0.0,-4.314044e-05,0.0,0.0,-4.314044e-05
